In [36]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1714164741602/work


In [39]:
import sklearn
print(sklearn.__version__)

1.4.2


In [37]:
!python -V

Python 3.9.19


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/opt/conda/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [22]:
year = 2023
month = 3
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet')

In [23]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [24]:
y_pred.std()

6.247488852238703

In [30]:
type(y_pred)

numpy.ndarray

In [25]:

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [32]:
predictionDf = pd.DataFrame(y_pred, columns=['prediction'])

In [33]:
df_result = pd.concat([df['ride_id'], predictionDf], axis=1)

In [35]:
df_result.to_parquet(
    'result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)